# March Madness

![Banner](./assets/March-Madness-generic-(Blue-Black).jpg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->

Guessing a perfect bracket is nearly impossible (1 in 120.2 billion odds if you know a little something about basketball). Winners of the Bracket Challenge Game averaged around 49.8% of correctly guessed games.

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->

Could machine learning make a perfect bracket (probably not) but could it beat the 49.8% average of correctly guessed games from the top players in the pool?  

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->

A machine learning model can beat the average winner rate of 49.8 correctly guessed games.

## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->

KenPom - a statistical archive of stats and ratings based on certain common metrics and custom statistical metrics (very popular among coaches, bookmakers, & bettors)

Basketball Reference - another statistical archive of college basketball games and stats, has most of the common statistics you are going to find in a game

Barttorvik - statistical archive of common stats and custom stats, like KenPom but has more custom metrics

Depedending on what stats/metrics I will be able to pull from the sources, I will relate the data based on team/year (regular season). Each data source has regular season stats/metrics for each year for each team. 

## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->

I will first formulate a method to determine how I will be able to include tournament data for each games/round. From there, I can start aggregating the team stats/metrics I want to test out in the data. Depending on the initial data analysis of these features, I will use some of them for the final model.

In [1]:
# Start your code here

## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->

https://www.ncaa.com/news/basketball-men/bracketiq/2022-03-10/perfect-ncaa-bracket-absurd-odds-march-madness-dream

https://kenpom.com/

https://www.basketball-reference.com/

https://barttorvik.com/#

In [2]:
# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python MarchMadness.ipynb

[NbConvertApp] Converting notebook source.ipynb to python
[NbConvertApp] Writing 1271 bytes to source.py
